In [1]:
from evaluation_runner import form_multiple_choice_prompt, random
import os
import pandas as pd
import json
from pathlib import Path

INFO 04-23 14:59:24 [__init__.py:239] Automatically detected platform cuda.


In [2]:
args_list = json.load(open('run_args_QWEN2.5_14B_AA_FT.json'))

In [3]:
def add_examples(num_examples, query_template, examples_method, examples_seed, df_train, system_prompt, labels):
    all_example_pairs = []
    random.seed(examples_seed)
    template_qa = "{sentence}\n### Question: Which is the most likely intent for this citation?\n{options}\n### Answer: "
    template_simple = "{sentence} \nClass: "
    multiple_choice = form_multiple_choice_prompt(labels) if query_template[0] == '2' else None
    prompt = template_qa if query_template[0] == '2' else template_simple

    if examples_method[0] == '1':
        all_example_pairs = [f"\n{prompt.format(sentence=ex, options=multiple_choice if multiple_choice else '')} {label}\n" for label in labels
            for ex in df_train[df_train['citation_class_label'] == label].sample(num_examples, random_state=examples_seed)['citation_context']]

        random.shuffle(all_example_pairs)
        all_example_pairs = ''.join(all_example_pairs)
        system_prompt[0]['content'] += "\n\n########\n\n# EXAMPLES #\n" + all_example_pairs

    elif examples_method[0] == '2':
        all_example_pairs = [
            [{"role": "user", "content": f"{prompt.format(sentence=ex, options=multiple_choice if multiple_choice else '')}"}, {"role": "assistant", "content": label}]
            for label in labels
            for ex in df_train[df_train['citation_class_label'] == label].sample(num_examples, random_state=examples_seed)['citation_context']
        ]
        random.shuffle(all_example_pairs)
        system_prompt.extend([pair for sublist in all_example_pairs for pair in sublist])

    return system_prompt

for args in args_list['args']:
    args['label_schema'] = args_list['label_schema']
    datasets_path = os.path.join('datasets', 'formatted')
    system_prompts_cfg = json.loads(Path('experimental-configs/system_prompts.json').read_text())
    prompt_id = f"{args['dataset']}{args['system_prompt']}"
    system_prompt = [{
            "role": "system",
            "content": system_prompts_cfg[prompt_id]
        }]
    df_train = pd.read_csv(os.path.join(datasets_path, args['dataset'], "train.csv"))
    with open('experimental-configs/experiments_cfg.json') as f:
            experiment_cfg = json.load(f)
            class_labels = experiment_cfg['class_labels']
    if args['method'] != 'zero-shot':
        prompts = add_examples(
            num_examples=args['num_examples'],
            query_template=args['query_template'],
            examples_method=args['example_method'],
            examples_seed=args['examples_seed'],
            df_train=df_train,
            system_prompt=system_prompt,
            labels=class_labels[args['dataset']]
        )

In [6]:
df_train['citation_class_label'].str.lower().to_list()

['uses',
 'background',
 'background',
 'background',
 'compares_contrasts',
 'background',
 'background',
 'compares_contrasts',
 'motivation',
 'compares_contrasts',
 'background',
 'background',
 'background',
 'extension',
 'background',
 'background',
 'background',
 'compares_contrasts',
 'background',
 'background',
 'uses',
 'background',
 'uses',
 'compares_contrasts',
 'background',
 'uses',
 'background',
 'motivation',
 'extension',
 'background',
 'future',
 'motivation',
 'compares_contrasts',
 'background',
 'background',
 'background',
 'background',
 'background',
 'background',
 'uses',
 'future',
 'uses',
 'background',
 'motivation',
 'motivation',
 'background',
 'uses',
 'background',
 'motivation',
 'motivation',
 'compares_contrasts',
 'uses',
 'background',
 'background',
 'uses',
 'background',
 'compares_contrasts',
 'background',
 'uses',
 'motivation',
 'compares_contrasts',
 'background',
 'compares_contrasts',
 'background',
 'motivation',
 'compares_cont